## Experiment objective
Is differencing technic effective?

## Experiment setting
compare the result with differencing and without differencing
### Dataset
train : AAPL, MSFT, NVDA, AMZN, COST stock close price (2000-01-01~2013-12-31)
test : AAPL, MSFT, NVDA, AMZN, COST stock close price (2014-01-01~2023-12-31)

### Scenario1
forecasting one-step ahead AAPL stock close price based on the past 23 steps on itself and 4 other stocks close price


In [1]:
import os
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.optim import Adam
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from torchmetrics import MeanAbsolutePercentageError


from data.dataloader import dataloader_info
from utils.utils import load_yaml_config, instantiate_from_config

from models.predictor import GRU
from models.solver import Trainer
from data.dataloader import dataloader_info
from utils.visualize import visualize_pca, visualize_tsne, visualize_kernel

/home/harim/Desktop/pyproject/Project_TS_Generation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def train_model(model, dataloader, criterion, optimizer, num_epochs, description, device):
    model.train()
    with tqdm(range(num_epochs), total=num_epochs) as pbar:
        for _ in pbar:
            for data_diff, *_ in dataloader:
                x_train = data_diff[:,:-1,:].float().to(device)
                y_train = data_diff[:,-1:,0].float().to(device)
                optimizer.zero_grad()
                outputs = model(x_train)
                loss = criterion(outputs, y_train)
                loss.backward()
                optimizer.step()
            pbar.set_description(f"{description} loss: {loss.item():.6f}")

In [3]:
def evaluate_model(model, dataloader, device):
    model.eval()
    l1loss = nn.L1Loss()
    l2loss = nn.MSELoss()
    mapeloss = MeanAbsolutePercentageError().to(device)
    
    total_l1 = 0
    total_l2 = 0
    predictions, ground_truth = [], []
    with torch.no_grad():
        for data_diff, data_norm, data_mean, data_std  in dataloader:
            data_norm = data_norm.to(device)
            data_diff = data_diff.to(device)
            data_mean = data_mean.to(device)
            data_std = data_std.to(device)
            batch_size = len(data_diff)
            x_diff = data_diff[:, :-1, :].float()
            y_true_diff = data_diff[:, -1:, :1].float()
            y_pred_diff = model(x_diff).view(-1,1,1)
            y_pred_norm = data_norm[:,-2:-1,:1] + y_pred_diff
            y_true_norm = data_norm[:,-2:-1,:1] + y_true_diff
            
            y_pred_unnorm = y_pred_norm * data_std[:, :, :1] + data_mean[:, :, :1]
            y_test_unnorm = y_true_norm * data_std[:, :, :1] + data_mean[:, :, :1]
            
            total_l1 += l1loss(y_pred_unnorm, y_test_unnorm) * batch_size
            total_l2 += l2loss(y_pred_unnorm, y_test_unnorm) * batch_size

            predictions.append(y_pred_unnorm.cpu().numpy())
            ground_truth.append(y_test_unnorm.cpu().numpy())

    n_data = len(dataloader.dataset)
    total_l1 /= n_data
    total_l2 /= n_data
    predictions = np.concatenate(predictions).squeeze()
    ground_truth = np.concatenate(ground_truth).squeeze()
    mape_loss = mapeloss(torch.tensor(predictions), torch.tensor(ground_truth)).item()
    
    return total_l1, total_l2, mape_loss, predictions, ground_truth

In [4]:
# Load configurations
configs = load_yaml_config("configs/experiments1.yaml")
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize Diffusion_TS Model
diffusion_ts = instantiate_from_config(configs['model']).to(device)
# diffusion_ts.load_state_dict(torch.load("check_points/stock/diff_23/model_diff_10000.pth"))
batch_size = 128

In [5]:
# load dataloader, dataset
dl_info_train = dataloader_info(configs, train=True)
dl_info_test = dataloader_info(configs, train=False)

dl_train = dl_info_train["dataloader"]
ds_train = dl_info_train["dataset"]

dl_test = dl_info_test["dataloader"]
ds_test = dl_info_test["dataset"]


In [6]:
trainer = Trainer(config_solver=configs["solver"], model=diffusion_ts, dataloader=dl_train)
trainer.train()

  0%|          | 0/20000 [00:00<?, ?it/s]/home/harim/Desktop/pyproject/Project_TS_Generation/.venv/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
combiend_loss: 0.038361 l1_loss : 0.417601 fourier_loss : 0.000000:  22%|██▏       | 4421/20000 [06:20<22:39, 11.46it/s]

In [7]:
# 1. pre-training for baseline predictor

predictor_base = GRU(input_dim=5, 
                   hidden_dim=50, 
                   output_dim=1, 
                   num_layers=2).to(device)
optimizer_base = Adam(predictor_base.parameters(), lr=0.001)
lossfn = nn.L1Loss()

train_model(predictor_base, 
            dl_train, 
            lossfn, 
            optimizer_base, 
            num_epochs=3000, 
            description="Baseline",
            device=device)

l1, l2, mape, pred_y, true_y = evaluate_model(predictor_base, dl_test, device=device)
print(f"Baseline : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")



Baseline loss: 0.018240: 100%|██████████| 3000/3000 [01:43<00:00, 29.12it/s]


Baseline : L1 loss: 1.28405 	 L2 Loss : 4.48231 	 MAPE loss : 0.01602 


In [ ]:
# only synthetic
syn_score = []
for e in range(5):
    # train a baseline predictor
    predictor_base = GRU(input_dim=5, hidden_dim=50, output_dim=1, num_layers=2).to(device)
    optimizer_base = Adam(predictor_base.parameters(), lr=0.001)
    lossfn = nn.L1Loss()
    train_model(predictor_base, 
                dl_train, 
                lossfn, 
                optimizer_base, 
                num_epochs=5000, 
                description="Baseline",
                device=device)
    l1, l2, mape, pred_y, true_y = evaluate_model(predictor_base, dl_test, device=device)
    print(f"Baseline : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")

    # additional training on synthetic data
    synthetic_data = diffusion_ts.generate_mts(batch_size=3000)
    dl_synthetic = DataLoader(synthetic_data, batch_size=batch_size, shuffle=True)
    train_model(predictor_base, 
                dl_synthetic, 
                lossfn, 
                optimizer_base, 
                num_epochs=5000, 
                description="Synthetic",
                device=device)
    l1, l2, mape, pred_y, true_y = evaluate_model(predictor_base, dl_test, device=device)
    syn_score.append([l1, l2, mape])
    print(f"Synthetic : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")

 loss: 0.019032: 100%|██████████| 3000/3000 [01:14<00:00, 40.25it/s]


original : L1 loss: 1.39391 	 L2 Loss : 5.13391 	 MAPE loss : 0.01688 


 loss: 0.012735: 100%|██████████| 5000/5000 [01:48<00:00, 46.29it/s]


synthetic_3000 : L1 loss: 1.21051 	 L2 Loss : 4.07728 	 MAPE loss : 0.01447 


 loss: 0.021471: 100%|██████████| 3000/3000 [01:14<00:00, 40.08it/s]


original : L1 loss: 1.40415 	 L2 Loss : 5.30638 	 MAPE loss : 0.01704 


reverse step from x_T to x_0:  32%|███▏      | 32/100 [00:20<00:42,  1.60it/s]


KeyboardInterrupt: 

(6000, 23, 5)

In [30]:
for e in range(5):
    gt_predictor = GRU(5, 50, 1, 2).to(device)
    gt_optimizer = Adam(gt_predictor.parameters(), lr=0.001)

    train_model(gt_predictor, dl_info["dataloader"],lossfn, gt_optimizer, num_epochs=3000)
    l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, test_loader)
    print(f"original : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")

    fake_data = []
    for _ in range(2):
        f_data = diffusion_ts.generate_mts(batch_size=3000)
        fake_data.append(f_data)
    fake_data = np.concatenate(fake_data)
    syn_dl = DataLoader(fake_data, batch_size=batch_size, shuffle=True)
    train_model(gt_predictor, syn_dl,lossfn, gt_optimizer, 5000)
    l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, test_loader)
    print(f"synthetic_3000 : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")



 loss: 0.017886: 100%|██████████| 3000/3000 [01:14<00:00, 40.19it/s]


original : L1 loss: 1.41482 	 L2 Loss : 5.32786 	 MAPE loss : 0.01695 


 loss: 0.019388: 100%|██████████| 5000/5000 [03:29<00:00, 23.83it/s]


synthetic_3000 : L1 loss: 1.26197 	 L2 Loss : 4.34497 	 MAPE loss : 0.01500 


 loss: 0.050949:  13%|█▎        | 380/3000 [00:09<01:05, 40.13it/s]


KeyboardInterrupt: 

In [31]:
for e in range(5):
    gt_predictor = GRU(5, 50, 1, 2).to(device)
    gt_optimizer = Adam(gt_predictor.parameters(), lr=0.001)

    train_model(gt_predictor, dl_info["dataloader"],lossfn, gt_optimizer, num_epochs=3000)
    l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, test_loader)
    print(f"original : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")

    fake_data = []
    for _ in range(3):
        f_data = diffusion_ts.generate_mts(batch_size=3000)
        fake_data.append(f_data)
    fake_data = np.concatenate(fake_data)
    syn_dl = DataLoader(fake_data, batch_size=batch_size, shuffle=True)
    train_model(gt_predictor, syn_dl,lossfn, gt_optimizer, 5000)
    l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, test_loader)
    print(f"synthetic_3000 : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")



 loss: 0.018860: 100%|██████████| 3000/3000 [01:14<00:00, 40.18it/s]


original : L1 loss: 1.42766 	 L2 Loss : 5.53979 	 MAPE loss : 0.01712 


 loss: 0.026771: 100%|██████████| 5000/5000 [05:14<00:00, 15.91it/s]


synthetic_3000 : L1 loss: 1.36441 	 L2 Loss : 5.14041 	 MAPE loss : 0.01632 


 loss: 0.047263:  10%|█         | 306/3000 [00:07<01:06, 40.43it/s]


KeyboardInterrupt: 

In [32]:
# ori + syn
for e in range(5):
    gt_predictor = GRU(5, 50, 1, 2).to(device)
    gt_optimizer = Adam(gt_predictor.parameters(), lr=0.001)

    train_model(gt_predictor, dl_info["dataloader"],lossfn, gt_optimizer, num_epochs=3000)
    l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, test_loader)
    print(f"original : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")

    idx = np.random.permutation(len(gt_data))[:1500]
    fake_data = diffusion_ts.generate_mts(batch_size=1500)
    syn_data = np.concatenate([gt_data[idx], fake_data])
    syn_dl = DataLoader(syn_data, batch_size=batch_size, shuffle=True)
    train_model(gt_predictor, syn_dl,lossfn,gt_optimizer, 5000)
    l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, test_loader)
    print(f"origin + synthetic : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")



 loss: 0.018162: 100%|██████████| 3000/3000 [01:15<00:00, 39.92it/s]


original : L1 loss: 1.38906 	 L2 Loss : 5.27340 	 MAPE loss : 0.01674 


reverse step from x_T to x_0:   0%|          | 0/100 [00:00<?, ?it/s]/home/harim/Desktop/pyproject/Project_TS_Generation/.venv/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
 loss: 0.013199: 100%|██████████| 5000/5000 [01:48<00:00, 45.99it/s]


origin + synthetic : L1 loss: 1.24834 	 L2 Loss : 4.38775 	 MAPE loss : 0.01485 


 loss: 0.054268:  10%|▉         | 292/3000 [00:07<01:07, 39.93it/s]


KeyboardInterrupt: 

In [33]:
# only origin
for e in range(5):
    gt_predictor = GRU(5, 50, 1, 2).to(device)
    gt_optimizer = Adam(gt_predictor.parameters(), lr=0.001)

    train_model(gt_predictor, dl_info["dataloader"],lossfn, gt_optimizer, num_epochs=3000)
    l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, test_loader)
    print(f"original : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")

    idx = np.random.permutation(len(gt_data))[:3000]
    syn_dl = DataLoader(gt_data[idx], batch_size=batch_size, shuffle=True)
    train_model(gt_predictor, syn_dl,lossfn,gt_optimizer, 5000)
    l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, test_loader)
    print(f"origin : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")



 loss: 0.017990: 100%|██████████| 3000/3000 [01:14<00:00, 40.40it/s]


original : L1 loss: 1.38049 	 L2 Loss : 5.06459 	 MAPE loss : 0.01658 


 loss: 0.011552: 100%|██████████| 5000/5000 [01:48<00:00, 45.89it/s]


origin : L1 loss: 1.29391 	 L2 Loss : 4.42872 	 MAPE loss : 0.01555 


 loss: 0.024289:  39%|███▉      | 1169/3000 [00:29<00:45, 40.25it/s]


KeyboardInterrupt: 

Adv_synthetic : L1 loss: 1.35900 	 L2 Loss : 4.97446 	 MAPE loss : 0.01662 

In [ ]:
gt_predictor = GRU(5, 50, 1, 2).to(device)
gt_optimizer = Adam(gt_predictor.parameters(), lr=0.001)

train_model(gt_predictor, dl_info["dataloader"],lossfn, gt_optimizer, num_epochs=3000)
l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, train_loader)
print(f"Adv_synthetic : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")


In [ ]:
for _ in range(50):
    train_model(gt_predictor, dl_info["dataloader"], lossfn, gt_optimizer, 2000)
    l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, test_loader)
    print(f"Adv_synthetic : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")



# not_diff

In [ ]:
import os
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.optim import Adam
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from torchmetrics import MeanAbsolutePercentageError


from data.dataloader import dataloader_info
from utils.utils import load_yaml_config, instantiate_from_config

from models.predictor import GRU
from models.solver import Trainer
from data.dataloader import dataloader_info
from utils.visualize import visualize_pca, visualize_tsne, visualize_kernel

import copy

In [ ]:
# Load configurations
configs = load_yaml_config("configs/stock_diff2.yaml")
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize Diffusion_TS Model
diffusion_ts = instantiate_from_config(configs['model']).to(device)
batch_size = 128

In [ ]:
# dataloader
dl_info = dataloader_info(configs)
dl_info_test = dataloader_info(configs, train=False)
dl_info_test["dataloader"]
dl_info_test["dataset"]

dataset = dl_info['dataset']

In [ ]:
dl = dl_info["dataloader"]

In [ ]:
diffusion_ts.load_state_dict(torch.load("check_points/stock/diff_24/model_notdiff_10000.pth"))


In [ ]:
# trainer = Trainer(config_solver=configs["solver"], model=diffusion_ts, dataloader=dl_info["dataloader"])
# trainer.train_decomp("model_notdiff")

In [ ]:
# generate adversarial data
fake_data = diffusion_ts.generate_mts(batch_size=1500)


In [ ]:
# load original data
seq_length, feature_dim = dataset.window, dataset.feature_dim
gt_data = np.load(os.path.join(dataset.dir, f"stock_origin_data_{seq_length}_train.npy"))
idx = np.random.permutation(len(gt_data))[:1500]
ori_data = gt_data[idx]

In [ ]:
visualize_pca(ori_data, fake_data, 1500)
visualize_tsne(ori_data, fake_data, 1500)
visualize_kernel(ori_data, fake_data, 1500)

In [ ]:
test_data_norm_origin = torch.from_numpy(np.load(os.path.join(dataset.dir, f"stock_origin_data_{seq_length}_test.npy"))).to(device)
test_mean = torch.from_numpy(np.load(os.path.join(dataset.dir, f"stock_origin_mean_{seq_length}_test.npy"))).to(device)
test_std = torch.from_numpy(np.load(os.path.join(dataset.dir, f"stock_origin_std_{seq_length}_test.npy"))).to(device)

test_dataset = TensorDataset(test_data_norm_origin, test_mean, test_std)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

lossfn = nn.L1Loss()
# lossfn = nn.MSELoss()


In [ ]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=100, description=""):
    model.train()
    with tqdm(range(num_epochs), total=num_epochs) as pbar:
        for e in pbar:
            for data in dataloader:
                x_train = data[:,:-1,:].float().to(device)
                y_train = data[:,-1:,0].float().to(device)
                optimizer.zero_grad()
                outputs = model(x_train)
                loss = criterion(outputs, y_train)
                loss.backward()
                optimizer.step()
            pbar.set_description(f"{description} loss: {loss.item():.6f}")
    

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()
    # define loss for comparison
    l1loss = nn.L1Loss()
    l2loss = nn.MSELoss()
    mapeloss = MeanAbsolutePercentageError().to(device)
    
    total_l1 = 0
    total_l2 = 0
    total_mape = 0

    predictions, true_vals = [], []
    with torch.no_grad():
        for data_norm, data_mean, data_std  in dataloader:
            x_test = data_norm[:, :(seq_length - 1), :].float().to(device)
            y_test = data_norm[:, (seq_length - 1):, :1].float().to(device)
            y_pred = model(x_test).view(-1,1,1)
            
            y_pred_unnorm = y_pred * data_std[:, :, :1] + data_mean[:, :, :1]
            y_test_unnorm = y_test * data_std[:, :, :1] + data_mean[:, :, :1]
            total_l1 += l1loss(y_pred_unnorm, y_test_unnorm) * len(data_norm)
            total_l2 += l2loss(y_pred_unnorm, y_test_unnorm) * len(data_norm)
            # total_mape += mapeloss(y_pred_unnorm, y_test_unnorm).item() * len(data_norm)

            predictions.append(y_pred_unnorm.cpu().numpy())
            true_vals.append(y_test_unnorm.cpu().numpy())

    n_data = len(dataloader.dataset)
    total_l1 /= n_data
    total_l2 /= n_data
    # total_mape /= n_data
    
    predictions = np.concatenate(predictions).squeeze()
    true_vals = np.concatenate(true_vals).squeeze()
    mape_loss = mapeloss(torch.tensor(predictions), torch.tensor(true_vals)).item()
    
    return total_l1, total_l2, mape_loss, predictions, true_vals

In [ ]:
gt_predictor = GRU(5, 50, 1, 2).to(device)
gt_optimizer = Adam(gt_predictor.parameters(), lr=0.001)

train_model(gt_predictor, dl_info["dataloader"],lossfn, gt_optimizer, num_epochs=3000)


In [ ]:
train_data_norm_origin = torch.from_numpy(np.load(os.path.join(dataset.dir, f"stock_origin_data_{seq_length}_train.npy"))).to(device)
train_mean = torch.from_numpy(np.load(os.path.join(dataset.dir, f"stock_origin_mean_{seq_length}_train.npy"))).to(device)
train_std = torch.from_numpy(np.load(os.path.join(dataset.dir, f"stock_origin_std_{seq_length}_train.npy"))).to(device)

train_dataset = TensorDataset(train_data_norm_origin, train_mean, train_std)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

lossfn = nn.L1Loss()
# lossfn = nn.MSELoss()


In [ ]:
l1, l2, mape, pre_y, true_y = evaluate_model(gt_predictor, train_loader)
print(f"Adv_synthetic : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")


In [ ]:
plt.plot(pre_y[-50:])
plt.plot(true_y[-50:])

In [ ]:
l1, l2, mape, pre_y, true_y = evaluate_model(gt_predictor, test_loader)
print(f"Adv_synthetic : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")


In [ ]:
plt.plot(pre_y[-50:], marker="o")
plt.plot(true_y[-50:], marker="o")

In [ ]:
gt_predictor = GRU(5, 50, 1, 2).to(device)
gt_optimizer = Adam(gt_predictor.parameters(), lr=0.001)

train_model(gt_predictor, dl_info["dataloader"],lossfn, gt_optimizer, num_epochs=3000)
l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, train_loader)
print(f"Adv_synthetic : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")


In [ ]:
for _ in range(50):
    idx = np.random.permutation(len(gt_data))[:1500]
    fake_data = diffusion_ts.generate_mts(batch_size=1500)
    syn_data = np.concatenate([gt_data[idx], fake_data])
    syn_dl = DataLoader(syn_data, batch_size=batch_size, shuffle=True)
    train_model(gt_predictor, syn_dl,lossfn,gt_optimizer, 5000)
    l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, test_loader)
    print(f"Adv_synthetic : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")



Adv_synthetic : L1 loss: 1.35900 	 L2 Loss : 4.97446 	 MAPE loss : 0.01662 

In [ ]:
gt_predictor = GRU(5, 50, 1, 2).to(device)
gt_optimizer = Adam(gt_predictor.parameters(), lr=0.001)

train_model(gt_predictor, dl_info["dataloader"],lossfn, gt_optimizer, num_epochs=3000)
l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, train_loader)
print(f"Adv_synthetic : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")


In [ ]:
for _ in range(50):
    train_model(gt_predictor, dl_info["dataloader"], lossfn, gt_optimizer, 2000)
    l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, test_loader)
    print(f"Adv_synthetic : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")



In [ ]:
gt_predictor = GRU(5, 50, 1, 2).to(device)
gt_optimizer = Adam(gt_predictor.parameters(), lr=0.001)

train_model(gt_predictor, dl_info["dataloader"],lossfn, gt_optimizer, num_epochs=3000)
l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, train_loader)
print(f"Adv_synthetic : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")


In [ ]:
for _ in range(50):
    fake_data = diffusion_ts.generate_mts(batch_size=3000)
    syn_dl = DataLoader(fake_data, batch_size=batch_size, shuffle=True)
    train_model(gt_predictor, syn_dl,lossfn, gt_optimizer, 5000)
    l1, l2, mape, pred_y, true_y = evaluate_model(gt_predictor, test_loader)
    print(f"Adv_synthetic : L1 loss: {l1:0.5f} \t L2 Loss : {l2:0.5f} \t MAPE loss : {mape:0.5f} ")

